In [1]:
from othello_rules import *
from othello_net import *
from tensorflow.python.framework import ops
from datetime import datetime
from example_states import *
from feature_extractor import *
from training_utils import *
import numpy as np


In [2]:
def avg_error(data_path, sess):
    errors = []
    validation_matches = get_all_matches(data_path)
    #XXX: Delete this line when testing is faster
    validation_matches = validation_matches[0:2]
    for i in range(len(validation_matches)):
        match = validation_matches[i]
        raw_match_movelist = match[8:]
        unpacked_movelist = unpack('b'*60, raw_match_movelist)
        board = initialize_game()
        player = -1
        previous_moves_avg = [56, 56, 56, 56]
        for move in unpacked_movelist:
            if move == 0:
                break
            feature_path = 'cache/validation/features/features_' + str(i) + "_" + str(move) + ".npy"
            label_path = 'cache/validation/labels/labels_' + str(i) + "_" + str(move) + ".npy"
            if not os.path.isfile(feature_path) and os.path.isfile(label_path):
                features = np.load(feature_path)
                label = np.load(label_path)
            else:
                features = board_to_input(board, player, previous_moves_avg)
                label = prepare_data(move_to_label(move))
                
            input_batch = [features]
            label_batch = [label]
            error = sess.run(loss, feed_dict={img_data:input_batch, ground_truths: label_batch, keep_prob:1.0, training:False})
            errors.append(error)
            board = make_move(board, move, player)
            if player is 1:
                player = -1
            else:
                player = 1
            legal_moves = find_legal_moves(board, player)
            if len(legal_moves) == 0:
                if player is 1:
                    player = -1
                else:
                    player = 1
            previous_moves_avg[3] = previous_moves_avg[2]
            previous_moves_avg[2] = previous_moves_avg[1]
            previous_moves_avg[1] = previous_moves_avg[0]
            previous_moves_avg[0] = move
            #input_batch = prepare_data(board * )
            #label_batch = prepare_data(move_to_label(move))
    return np.sum(errors) / len(errors)

def prediction_accuracy(data_path, len_games=2):
    lengths = []
    successes = []
    validation_matches = get_all_matches(data_path)
    #XXX: Delete this line when testing is faster
    validation_matches = validation_matches[0:len_games]
    for i in range(len_games):
        test_match = validation_matches[i]
        board = initialize_game()
        #print(board)
        #print('\n')
        player = -1
        success = 0
        length = 0
        #test_match = matches[i]
        raw_match_movelist = test_match[8:]
        unpacked_movelist = unpack('b'*60, raw_match_movelist)
        previous_moves_pred = [56, 56, 56, 56]
        for move in unpacked_movelist:
            length += 1
            if move == 0:
                winner = get_winner(board, 1, -1)
                break

            feature_path = 'cache/validation/features/features_' + str(i) + "_" + str(move) + ".npy"
            label_path = 'cache/validation/labels/labels_' + str(i) + "_" + str(move) + ".npy"
            if not os.path.isfile(feature_path) and os.path.isfile(label_path):
                features = np.load(feature_path)
                label = np.load(label_path)
            else:
                features = board_to_input(board, player, previous_moves_pred)
                label = prepare_data(move_to_label(move))
                
            input_batch = [features]
            label_batch = [label]
            prediction = sess.run(pred_up, feed_dict={img_data:input_batch, ground_truths: label_batch, keep_prob:1.0, training:False})
            np.set_printoptions(precision=2)
            prediction = np.transpose(prediction[0])
            prediction = np.transpose(prediction[1])
            legal_moves = find_legal_moves(board, player)
            cleaned_predictions = zero_illegal_moves(prediction, legal_moves)
            i,j = np.unravel_index(cleaned_predictions.argmax(), cleaned_predictions.shape)
            move_argmax = str((i+1) * 10 + (j+1))
            if str(move) == str(move_argmax):
                success += 1
            original_board = np.array(board)
            board_upright = np.transpose(original_board)
            board_upleft = np.rot90(np.rot90(board_upright))
            board_both_flips = np.transpose(board_upleft)
            if np.array_equal(board, board_upright):
                if str(move_argmax) == flip_move_upright(move):
                    success += 1
            if np.array_equal(board, board_upleft):
                if str(move_argmax) == flip_move_upleft(move):
                    success += 1
            if np.array_equal(board, board_both_flips):
                if str(move_argmax) == flip_move_upright(flip_move_upleft(move)):
                    success += 1
            board = make_move(board, move, player)
            if player is 1:
                player = -1
            else:
                player = 1
            legal_moves = find_legal_moves(board, player)
            if len(legal_moves) == 0:
                if player is 1:
                    player = -1
                else:
                    player = 1
        previous_moves_pred[3] = previous_moves_pred[2]
        previous_moves_pred[2] = previous_moves_pred[1]
        previous_moves_pred[1] = previous_moves_pred[0]
        previous_moves_pred[0] = move
        legal_moves = find_legal_moves(board, player)
        winner = get_winner(board, 1, -1)
        successes.append(success)
        lengths.append(length)
    
    return np.mean(successes)

In [3]:
# Ræsum graphið fyrir tensorflow
ops.reset_default_graph()
graph, img_data, train_step, optimizer, ground_truths, loss, pred_up, keep_prob, learn_rate, score_out, training = create_othello_net()
saver = tf.train.Saver()
sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))
init_op = tf.global_variables_initializer()
sess.run(init_op)
current_model = "models/deleteme_4.ckpt"
#if os.path.isfile(current_model):
#    saver.restore(sess, current_model)
validation_path = "validation/"
matches = get_all_matches('training/')
lenmatches = len(matches)
print(lenmatches)

# Byrjum þjálfunina
print("start training")
#print("starting error:" + str(avg_error(validation_path, sess)))
#print('%s: Step %d: Prediction accuracy = %.2f' % (datetime.now(), 0,
#                                                      prediction_accuracy()/float(60)))
iterations = 500
prev_stop = 0
probs = 1
for i in range(prev_stop, prev_stop+iterations):
    #TODO: Skrifa þetta fall
    #input_batch, label_batch = prepare_train_batch(train_ids, batch_size, do_flips, do_rots, data_path)
    current_match = matches[i]
    raw_match_movelist = current_match[8:]
    unpacked_movelist = unpack('b'*60, raw_match_movelist)
    
    board = initialize_game()
    
    training_stability = np.zeros((8,8))
    player = -1
    
    # One training batch is all the data from one match
    input_batch = []
    label_batch = []
    previous_moves = [56, 56, 56, 56]
    for move in unpacked_movelist:
        if move == 0:
            break
        # TODO: Athuga hvernig rotation sé löglegt og bæta þeim svo við
        feature_path = 'cache/training/features/features_' + str(i) + "_" + str(move) + ".npy"
        label_path = 'cache/training/labels/labels_' + str(i) + "_" + str(move) + ".npy"
        #XXX: Virðist vera helvíti böggað ef ég tek 'not' í burtu frá hér fyrir neðan!
        if not os.path.isfile(feature_path) and os.path.isfile(label_path):
            try:
                features = np.load(feature_path)
                label = np.load(label_path)
            except:
                print("data corruption in match " + str(i))
                features = board_to_input(board, player, previous_moves)
                label = prepare_data(move_to_label(move))
        else:
            features = board_to_input(board, player, previous_moves)
            label = prepare_data(move_to_label(move))
        input_batch.append(features)
        label_batch.append(label)
        # Now we add 3 reflections of the game state
        # which is done by flipping the board over one diagonal        
        move_upright = flip_move_upright(move)
        features_upright = flip_features(features, 'upright')
        input_batch.append(features_upright)
        label_batch.append(prepare_data(move_to_label(move_upright)))
        
        # Then the other diagonal
        move_upleft = flip_move_upleft(move)
        features_upleft = flip_features(features, 'upleft')
        input_batch.append(features_upleft)
        label_batch.append(prepare_data(move_to_label(move_upleft)))
        
        # Then both diagonals
        move_both = flip_move_upright(flip_move_upleft(move))
        features_both = flip_features(features, 'both')
        input_batch.append(features_both)
        label_batch.append(prepare_data(move_to_label(move_both)))
        
        
        board = make_move(board, move, player)
        
        if player is 1:
            player = -1
        else:
            player = 1
        legal_moves = find_legal_moves(board, player)
        if len(legal_moves) == 0:
            if player is 1:
                player = -1
            else:
                player = 1
        previous_moves[3] = previous_moves[2]
        previous_moves[2] = previous_moves[1]
        previous_moves[1] = previous_moves[0]
        previous_moves[0] = move
    
        
    start = 6e-4
    eta = start
    train_step.run(session=sess, feed_dict={img_data:input_batch,
                                            ground_truths: label_batch,
                                            keep_prob:probs,
                                            learn_rate:eta,
                                               training:True})

    if (i % 10 is 0) and (i > 0 and i < 51) or (i+1) == (iterations+prev_stop) or (i % 50 is 0):  
        print('%s, Step %d, Accuracy = %.3f, Loss = %.3f' % (datetime.now().strftime("%d. %b %H:%M:%S"), i,
                                                      prediction_accuracy(validation_path)/float(60),
                                                                          avg_error(validation_path, sess)))
        save_path = saver.save(sess, current_model)

print("done")

117298
start training
27. Jan 18:03:14, Step 0, Accuracy = 0.175, Loss = 42.636
27. Jan 18:03:24, Step 10, Accuracy = 0.200, Loss = 17.416
27. Jan 18:03:33, Step 20, Accuracy = 0.183, Loss = 6.451
27. Jan 18:03:42, Step 30, Accuracy = 0.183, Loss = 7.009


KeyboardInterrupt: 

In [4]:
# Try inspecting the output of the network
import matplotlib.pyplot as plt
np.set_printoptions(threshold=np.inf)
validation_path = "validation/"
#print("number of correct guesses: " + str(prediction_accuracy(validation_path)))
print("Prediction accuracy: " + str(prediction_accuracy(validation_path)/float(60)))

Prediction accuracy: 0.183333333333


decay = 0.9
117298
start training
26. Jan 16:51:29, Step 0, Accuracy = 0.183, Loss = 33.274
26. Jan 16:51:36, Step 10, Accuracy = 0.225, Loss = 26.724
26. Jan 16:51:44, Step 20, Accuracy = 0.217, Loss = 20.746
26. Jan 16:51:53, Step 30, Accuracy = 0.225, Loss = 15.878
26. Jan 16:52:00, Step 40, Accuracy = 0.208, Loss = 11.551
26. Jan 16:52:09, Step 50, Accuracy = 0.200, Loss = 8.413
26. Jan 16:52:37, Step 100, Accuracy = 0.167, Loss = 5.688
26. Jan 16:53:06, Step 150, Accuracy = 0.183, Loss = 5.371
26. Jan 16:53:31, Step 200, Accuracy = 0.200, Loss = 5.380
26. Jan 16:53:58, Step 250, Accuracy = 0.208, Loss = 5.169
26. Jan 16:54:22, Step 300, Accuracy = 0.200, Loss = 5.138
26. Jan 16:54:47, Step 350, Accuracy = 0.217, Loss = 5.084
26. Jan 16:55:13, Step 400, Accuracy = 0.233, Loss = 5.063

No BN
117298
start training
26. Jan 18:55:08, Step 0, Accuracy = 0.258, Loss = 32.628
26. Jan 18:55:19, Step 10, Accuracy = 0.208, Loss = 7.786
26. Jan 18:55:30, Step 20, Accuracy = 0.225, Loss = 6.673
26. Jan 18:55:40, Step 30, Accuracy = 0.225, Loss = 6.680
26. Jan 18:55:49, Step 40, Accuracy = 0.200, Loss = 5.988
26. Jan 18:55:59, Step 50, Accuracy = 0.217, Loss = 5.724
26. Jan 18:56:42, Step 100, Accuracy = 0.242, Loss = 4.360
26. Jan 18:57:30, Step 150, Accuracy = 0.300, Loss = 2.973
26. Jan 18:58:19, Step 200, Accuracy = 0.383, Loss = 2.706
26. Jan 18:59:07, Step 250, Accuracy = 0.425, Loss = 2.606
26. Jan 18:59:52, Step 300, Accuracy = 0.408, Loss = 2.493
26. Jan 19:00:48, Step 350, Accuracy = 0.408, Loss = 2.487
26. Jan 19:01:41, Step 400, Accuracy = 0.408, Loss = 2.480
26. Jan 19:02:35, Step 450, Accuracy = 0.408, Loss = 2.478
26. Jan 19:03:31, Step 499, Accuracy = 0.408, Loss = 2.493
done